In [1]:
#Generic Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Data Split Libraries
import sklearn
from sklearn.model_selection import train_test_split
#Tabulating Data
from tabulate import tabulate
#Garbage
import gc

In [2]:

#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession
#Apache Spark ML CLassifier Libraries
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier,NaiveBayes
#Apache Spark Evaluation Library
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#Apache Spark Features libraries
from pyspark.ml.feature import StandardScaler,StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder

#Apache Spark Pipelin Library
from pyspark.ml import Pipeline
# Apache Spark `DenseVector`
from pyspark.ml.linalg import DenseVector


In [3]:
#import pyspark

In [4]:
#from pyspark.sql import SparkSession

In [5]:
import cml.data_v1 as cmldata

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "jqrhv9chljtm"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()


Setting spark.hadoop.yarn.resourcemanager.principal to csso_jprosser


Spark Application Id:spark-application-1727206986973


Hive Session ID = ba851a6d-3e37-4bda-a1a1-ae7d06affc6a


+------------------+
|         namespace|
+------------------+
|           default|
|information_schema|
|               sys|
+------------------+



In [6]:
##spark = SparkSession.builder.appName("Practice_1").getOrCreate()

In [7]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [8]:
data_frame = [("Ahmed","","AlHashmi","201210044","Male",20000),
    ("Mariam","Hasan","Almenhali","201455411","Female",42000),
    ("Amna","Salem","Alameri","202010222","Female",16000),
    ("Zayed","Ali","AlZaabi","201788695","Male",26300),
    ("Maitha","Saeed","Alhosani","204974582","Female",""),
    ("Shatha","Abdalqader","Alharthi","201788695","Female",70600),
    ("Abdulla","Said","Almessabi","2011425698","Male",50300),
    ("Layala","Hamad","AlShamsi","","Female",38300)
  ]

In [9]:
schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True) \
  ])


In [10]:
df = spark.createDataFrame(data=data_frame,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



+---------+----------+---------+----------+------+------+
|firstname|middlename|lastname |id        |gender|salary|
+---------+----------+---------+----------+------+------+
|Ahmed    |          |AlHashmi |201210044 |Male  |20000 |
|Mariam   |Hasan     |Almenhali|201455411 |Female|42000 |
|Amna     |Salem     |Alameri  |202010222 |Female|16000 |
|Zayed    |Ali       |AlZaabi  |201788695 |Male  |26300 |
|Maitha   |Saeed     |Alhosani |204974582 |Female|      |
|Shatha   |Abdalqader|Alharthi |201788695 |Female|70600 |
|Abdulla  |Said      |Almessabi|2011425698|Male  |50300 |
|Layala   |Hamad     |AlShamsi |          |Female|38300 |
+---------+----------+---------+----------+------+------+



In [11]:
df.filter(df.firstname.startswith("A")).show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|    Ahmed|          | AlHashmi| 201210044|  Male| 20000|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|
+---------+----------+---------+----------+------+------+



In [12]:
df.filter(df.id.endswith("2")).show()

+---------+----------+--------+---------+------+------+
|firstname|middlename|lastname|       id|gender|salary|
+---------+----------+--------+---------+------+------+
|     Amna|     Salem| Alameri|202010222|Female| 16000|
|   Maitha|     Saeed|Alhosani|204974582|Female|      |
+---------+----------+--------+---------+------+------+



In [13]:
df.filter(df.lastname.contains("m")).show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|    Ahmed|          | AlHashmi| 201210044|  Male| 20000|
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|
|   Layala|     Hamad| AlShamsi|          |Female| 38300|
+---------+----------+---------+----------+------+------+



In [14]:
from pyspark.sql.functions import col,lit 
df.withColumn("Nationality", lit("UAE")).show()

+---------+----------+---------+----------+------+------+-----------+
|firstname|middlename| lastname|        id|gender|salary|Nationality|
+---------+----------+---------+----------+------+------+-----------+
|    Ahmed|          | AlHashmi| 201210044|  Male| 20000|        UAE|
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|        UAE|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|        UAE|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|        UAE|
|   Maitha|     Saeed| Alhosani| 204974582|Female|      |        UAE|
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|        UAE|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|        UAE|
|   Layala|     Hamad| AlShamsi|          |Female| 38300|        UAE|
+---------+----------+---------+----------+------+------+-----------+



In [15]:
newRow = spark.createDataFrame([("Khaled","Ali","Almasood","201652447","Male",36944)])
new_df = df.union(newRow)
new_df.show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|    Ahmed|          | AlHashmi| 201210044|  Male| 20000|
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|
|   Maitha|     Saeed| Alhosani| 204974582|Female|      |
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|
|   Layala|     Hamad| AlShamsi|          |Female| 38300|
|   Khaled|       Ali| Almasood| 201652447|  Male| 36944|
+---------+----------+---------+----------+------+------+



In [16]:
newRow = spark.createDataFrame([("Reem","Buthaina","Zayed","202499999","Female",10000)])
new_df = new_df.union(newRow)
new_df.show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|    Ahmed|          | AlHashmi| 201210044|  Male| 20000|
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|
|   Maitha|     Saeed| Alhosani| 204974582|Female|      |
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|
|   Layala|     Hamad| AlShamsi|          |Female| 38300|
|   Khaled|       Ali| Almasood| 201652447|  Male| 36944|
|     Reem|  Buthaina|    Zayed| 202499999|Female| 10000|
+---------+----------+---------+----------+------+------+



In [17]:
from pyspark.sql.functions import when
new_df.withColumn("levels",
    when((new_df.salary < 18000), lit("workers"))
     .when ((new_df.salary > 18000) & (new_df.salary <=38000), lit("senior"))             
       .otherwise(lit("executive"))
                 ).show()

+---------+----------+---------+----------+------+------+---------+
|firstname|middlename| lastname|        id|gender|salary|   levels|
+---------+----------+---------+----------+------+------+---------+
|    Ahmed|          | AlHashmi| 201210044|  Male| 20000|   senior|
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|executive|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|  workers|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|   senior|
|   Maitha|     Saeed| Alhosani| 204974582|Female|      |executive|
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|executive|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|executive|
|   Layala|     Hamad| AlShamsi|          |Female| 38300|executive|
|   Khaled|       Ali| Almasood| 201652447|  Male| 36944|   senior|
|     Reem|  Buthaina|    Zayed| 202499999|Female| 10000|  workers|
+---------+----------+---------+----------+------+------+---------+



In [18]:
from pyspark.sql.functions import col,isnan,when,count
df2 = new_df.select([count(when(col(c).contains('None') |
                            col(c).contains('NULL') |
                               (col(c) == '') |
                               col(c).isNull() |
                               isnan(c), c
                               )).alias(c)
                    for c in df.columns])
df2.show()

+---------+----------+--------+---+------+------+
|firstname|middlename|lastname| id|gender|salary|
+---------+----------+--------+---+------+------+
|        0|         1|       0|  1|     0|     1|
+---------+----------+--------+---+------+------+



In [19]:
from pyspark.sql.functions import col,when
df2=new_df.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in df.columns])
df2.show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|    Ahmed|      null| AlHashmi| 201210044|  Male| 20000|
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|
|   Maitha|     Saeed| Alhosani| 204974582|Female|  null|
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|
|   Layala|     Hamad| AlShamsi|      null|Female| 38300|
|   Khaled|       Ali| Almasood| 201652447|  Male| 36944|
|     Reem|  Buthaina|    Zayed| 202499999|Female| 10000|
+---------+----------+---------+----------+------+------+



In [20]:
#delete all missing values from all attributes
df3 = df2.na.drop()
df3.show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|
|   Khaled|       Ali| Almasood| 201652447|  Male| 36944|
|     Reem|  Buthaina|    Zayed| 202499999|Female| 10000|
+---------+----------+---------+----------+------+------+



In [21]:
from pyspark.sql.functions import when
df3.withColumn("levels",
    when((df3.salary < 18000), lit("workers"))
     .when ((df3.salary > 18000) & (df3.salary <=38000), lit("senior"))             
       .otherwise(lit("executive"))
                 ).show()

+---------+----------+---------+----------+------+------+---------+
|firstname|middlename| lastname|        id|gender|salary|   levels|
+---------+----------+---------+----------+------+------+---------+
|   Mariam|     Hasan|Almenhali| 201455411|Female| 42000|executive|
|     Amna|     Salem|  Alameri| 202010222|Female| 16000|  workers|
|    Zayed|       Ali|  AlZaabi| 201788695|  Male| 26300|   senior|
|   Shatha|Abdalqader| Alharthi| 201788695|Female| 70600|executive|
|  Abdulla|      Said|Almessabi|2011425698|  Male| 50300|executive|
|   Khaled|       Ali| Almasood| 201652447|  Male| 36944|   senior|
|     Reem|  Buthaina|    Zayed| 202499999|Female| 10000|  workers|
+---------+----------+---------+----------+------+------+---------+



In [22]:
df3.sort(col("firstname")).show(truncate=False)

+---------+----------+---------+----------+------+------+
|firstname|middlename|lastname |id        |gender|salary|
+---------+----------+---------+----------+------+------+
|Abdulla  |Said      |Almessabi|2011425698|Male  |50300 |
|Amna     |Salem     |Alameri  |202010222 |Female|16000 |
|Khaled   |Ali       |Almasood |201652447 |Male  |36944 |
|Mariam   |Hasan     |Almenhali|201455411 |Female|42000 |
|Reem     |Buthaina  |Zayed    |202499999 |Female|10000 |
|Shatha   |Abdalqader|Alharthi |201788695 |Female|70600 |
|Zayed    |Ali       |AlZaabi  |201788695 |Male  |26300 |
+---------+----------+---------+----------+------+------+



In [23]:
df3.orderBy(col("salary").asc()).show(truncate=False)

+---------+----------+---------+----------+------+------+
|firstname|middlename|lastname |id        |gender|salary|
+---------+----------+---------+----------+------+------+
|Reem     |Buthaina  |Zayed    |202499999 |Female|10000 |
|Amna     |Salem     |Alameri  |202010222 |Female|16000 |
|Zayed    |Ali       |AlZaabi  |201788695 |Male  |26300 |
|Khaled   |Ali       |Almasood |201652447 |Male  |36944 |
|Mariam   |Hasan     |Almenhali|201455411 |Female|42000 |
|Abdulla  |Said      |Almessabi|2011425698|Male  |50300 |
|Shatha   |Abdalqader|Alharthi |201788695 |Female|70600 |
+---------+----------+---------+----------+------+------+



In [24]:
#Sort by Descending (DESC)
df3.sort(col("id").desc()).show(truncate=False)

+---------+----------+---------+----------+------+------+
|firstname|middlename|lastname |id        |gender|salary|
+---------+----------+---------+----------+------+------+
|Reem     |Buthaina  |Zayed    |202499999 |Female|10000 |
|Amna     |Salem     |Alameri  |202010222 |Female|16000 |
|Shatha   |Abdalqader|Alharthi |201788695 |Female|70600 |
|Zayed    |Ali       |AlZaabi  |201788695 |Male  |26300 |
|Khaled   |Ali       |Almasood |201652447 |Male  |36944 |
|Mariam   |Hasan     |Almenhali|201455411 |Female|42000 |
|Abdulla  |Said      |Almessabi|2011425698|Male  |50300 |
+---------+----------+---------+----------+------+------+



In [25]:
#Update Column Based on Condition
from pyspark.sql.functions import when
df3 = df3.withColumn("gender", when(df3.gender == "Male","M")
       .when(df3.gender == "Female","F")
        .otherwise(df2.gender))
df3.show()

+---------+----------+---------+----------+------+------+
|firstname|middlename| lastname|        id|gender|salary|
+---------+----------+---------+----------+------+------+
|   Mariam|     Hasan|Almenhali| 201455411|     F| 42000|
|     Amna|     Salem|  Alameri| 202010222|     F| 16000|
|    Zayed|       Ali|  AlZaabi| 201788695|     M| 26300|
|   Shatha|Abdalqader| Alharthi| 201788695|     F| 70600|
|  Abdulla|      Said|Almessabi|2011425698|     M| 50300|
|   Khaled|       Ali| Almasood| 201652447|     M| 36944|
|     Reem|  Buthaina|    Zayed| 202499999|     F| 10000|
+---------+----------+---------+----------+------+------+



In [26]:
#Practice 2
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Practice2").getOrCreate()

In [27]:
df_pyspark=spark.read.csv("/home/cdsw/Loan_Default.csv",header=True,inferSchema=True)

In [28]:
df_pyspark.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- loan_limit: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- approv_in_adv: string (nullable = true)
 |-- loan_type: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- Credit_Worthiness: string (nullable = true)
 |-- open_credit: string (nullable = true)
 |-- business_or_commercial: string (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- rate_of_interest: double (nullable = true)
 |-- Interest_rate_spread: double (nullable = true)
 |-- Upfront_charges: double (nullable = true)
 |-- term: double (nullable = true)
 |-- Neg_ammortization: string (nullable = true)
 |-- interest_only: string (nullable = true)
 |-- lump_sum_payment: string (nullable = true)
 |-- property_value: double (nullable = true)
 |-- construction_type: string (nullable = true)
 |-- occupancy_type: string (nullable = true)
 |-- Secured_by: string (nullable = true)
 |-- total_unit

In [29]:
df_pyspark.show()

+-----+----+----------+-----------------+-------------+---------+------------+-----------------+-----------+----------------------+-----------+----------------+--------------------+---------------+-----+-----------------+-------------+----------------+--------------+-----------------+--------------+----------+-----------+-------+-----------+------------+------------------------+-----+-------------------------+-----------+-------+-------------+------+-----+
|   ID|year|loan_limit|           Gender|approv_in_adv|loan_type|loan_purpose|Credit_Worthiness|open_credit|business_or_commercial|loan_amount|rate_of_interest|Interest_rate_spread|Upfront_charges| term|Neg_ammortization|interest_only|lump_sum_payment|property_value|construction_type|occupancy_type|Secured_by|total_units| income|credit_type|Credit_Score|co-applicant_credit_type|  age|submission_of_application|        LTV| Region|Security_Type|Status|dtir1|
+-----+----+----------+-----------------+-------------+---------+------------+

In [30]:
df_pyspark.groupBy('Gender').max().show()

+-----------------+-------+---------+----------------+---------------------+-------------------------+--------------------+---------+-------------------+-----------+-----------------+-----------+-----------+----------+
|           Gender|max(ID)|max(year)|max(loan_amount)|max(rate_of_interest)|max(Interest_rate_spread)|max(Upfront_charges)|max(term)|max(property_value)|max(income)|max(Credit_Score)|   max(LTV)|max(Status)|max(dtir1)|
+-----------------+-------+---------+----------------+---------------------+-------------------------+--------------------+---------+-------------------+-----------+-----------------+-----------+-----------+----------+
|Sex Not Available| 173555|     2019|         3346500|                  7.5|                   2.5932|            37604.38|    360.0|           1.0008E7|   578580.0|              900|263.5416667|          1|      61.0|
|           Female| 173559|     2019|         2056500|                  6.5|                   2.0022|            53485.78| 

In [31]:
df_pyspark.groupBy('loan_type').sum().show()

+---------+-----------+---------+----------------+---------------------+-------------------------+--------------------+----------+-------------------+-----------+-----------------+------------------+-----------+----------+
|loan_type|    sum(ID)|sum(year)|sum(loan_amount)|sum(rate_of_interest)|sum(Interest_rate_spread)|sum(Upfront_charges)| sum(term)|sum(property_value)|sum(income)|sum(Credit_Score)|          sum(LTV)|sum(Status)|sum(dtir1)|
+---------+-----------+---------+----------------+---------------------+-------------------------+--------------------+----------+-------------------+-----------+-----------------+------------------+-----------+----------+
|    type3| 1461512713| 29749965|      5007887500|   39805.130000000325|      -1007.0949000000019| 2.915294928000004E7| 5155546.0|          5.32526E9| 4.839204E7|         10289460|1114599.6120457398|       3692|  313740.0|
|    type1|11227716479|228496287|     38884764500|    359664.7499999796|       37077.368299999995| 2.9694270

In [32]:
df_pyspark.groupBy('loan_amount').mean().show()

+-----------+------------------+---------+----------------+---------------------+-------------------------+--------------------+------------------+-------------------+------------------+-----------------+------------------+-------------------+------------------+
|loan_amount|           avg(ID)|avg(year)|avg(loan_amount)|avg(rate_of_interest)|avg(Interest_rate_spread)|avg(Upfront_charges)|         avg(term)|avg(property_value)|       avg(income)|avg(Credit_Score)|          avg(LTV)|        avg(Status)|        avg(dtir1)|
+-----------+------------------+---------+----------------+---------------------+-------------------------+--------------------+------------------+-------------------+------------------+-----------------+------------------+-------------------+------------------+
|     586500|  96248.1960375391|   2019.0|        586500.0|    3.944122137404581|      0.21576068702290088|  3416.7005598958335| 351.4786235662148|  829804.7673098751| 10623.55353075171|696.6308654848801| 75.147

In [33]:
df_pyspark.groupBy('loan_purpose').count().show()

+------------+-----+
|loan_purpose|count|
+------------+-----+
|        null|  134|
|          p2| 3274|
|          p1|34529|
|          p3|55934|
|          p4|54799|
+------------+-----+



In [34]:
df_pyspark.agg({'loan_amount' : 'sum'}).show()

+----------------+
|sum(loan_amount)|
+----------------+
|     49227275000|
+----------------+



In [35]:
import pyspark.sql.functions as fn

df_pyspark.agg(
    fn.count('Region').alias('count'),
    fn.countDistinct('Region').alias('distinct')
).show()

+------+--------+
| count|distinct|
+------+--------+
|148670|       4|
+------+--------+

